In [1]:
import numpy as np 
import pandas as pd 

In [2]:
df = pd.read_csv("C:/Program Files/PostgreSQL/16/data/credit_card.csv")

In [3]:
#clean column names 
df.columns = df.columns.str.strip().str.lower()
df.columns

Index(['client_num', 'card_category', 'annual_fees', 'activation_30_days',
       'customer_acq_cost', 'week_start_date', 'week_num', 'qtr',
       'current_year', 'credit_limit', 'total_revolving_bal',
       'total_trans_amt', 'total_trans_vol', 'avg_utilization_ratio',
       'use chip', 'exp type', 'interest_earned', 'delinquent_acc'],
      dtype='object')

In [4]:
#clean whitespaces in columns
for i in df.select_dtypes(include=['object']).columns:
    df[i] = df[i].astype(str).str.strip()

In [5]:
#to clean currency and related symbols from columns 
for col in df.columns:
    # sirf object/string type columns pe try karenge
    if df[col].dtype == "object":
        # 1) clean string values: comma, %, ₹, $, spaces hatao
        clean = (
            df[col]
            .astype(str)
            .str.replace(r"[,%₹$]", "", regex=True)
            .str.replace(r"\s+", "", regex=True)
        )

In [6]:
#normalize NA values 
import numpy as np

na_values = ["na", "n/a", "none", "null", ""]

for col in df.columns:
    if df[col].dtype == "object":
        df[col] = df[col].replace(na_values, np.nan)


In [7]:
# standardizing datetime columns
dates = [col for col in df.columns 
         if any(key.lower() in col.lower() 
                for key in ["date","time","dt","timestamp"])]

print(dates)
# -> ['week_start_date']

['week_start_date']


In [8]:
col = 'week_start_date'

try:
    df[col] = pd.to_datetime(df[col], errors='raise')
    print(f"Converted: {col}, dtype -> {df[col].dtype}")
except Exception as e:
    print(f"Failed converting {col}: {e}")
    print(df[col].head())


Failed converting week_start_date: time data "15-01-2023" doesn't match format "%m-%d-%Y", at position 2. You might want to try:
    - passing `format` if your strings have a consistent format;
    - passing `format='ISO8601'` if your strings are all ISO8601 but not necessarily in exactly the same format;
    - passing `format='mixed'`, and the format will be inferred for each element individually. You might want to use `dayfirst` alongside this.
0    01-01-2023
1    01-01-2023
2    01-01-2023
3    01-01-2023
4    01-01-2023
Name: week_start_date, dtype: object


In [9]:
#create parts of dates from datetimes cols 
for col in dates:
    if pd.api.types.is_datetime64_any_dtype(df[col]):
        base = col + ""   # keep original name
        df[f"{base}_year"] = df[col].dt.year
        df[f"{base}_month"] = df[col].dt.month
        df[f"{base}_day"] = df[col].dt.day


In [10]:
#handling missing values
for col in df.columns:

    # Numeric columns → median
    if pd.api.types.is_numeric_dtype(df[col]):
        df[col] = df[col].fillna(df[col].median())

    # Datetime columns → mode
    elif pd.api.types.is_datetime64_any_dtype(df[col]):
        if not df[col].mode().empty:
            df[col] = df[col].fillna(df[col].mode()[0])
        else:
            df[col] = df[col].ffill().bfill()

    # Object / categorical → forward + backward fill
    else:
        df[col] = df[col].ffill().bfill()

In [11]:
df.head(5)

,client_num,card_category,annual_fees,activation_30_days,customer_acq_cost,week_start_date,week_num,qtr,current_year,credit_limit,total_revolving_bal,total_trans_amt,total_trans_vol,avg_utilization_ratio,use chip,exp type,interest_earned,delinquent_acc
0,708082083,Blue,200,0,87,01-01-2023,Week-1,Q1,2023,3544.0,1661,15149,111,0.469,Chip,Travel,4393.21,0
1,708083283,Blue,445,1,108,01-01-2023,Week-1,Q1,2023,3421.0,2517,992,21,0.736,Swipe,Entertainment,69.44,0
2,708084558,Blue,140,0,106,01-01-2023,Week-1,Q1,2023,8258.0,1771,1447,23,0.214,Chip,Bills,202.58,0
3,708085458,Blue,250,1,150,01-01-2023,Week-1,Q1,2023,1438.3,0,3940,82,0.000,Online,Grocery,236.40,0
4,708086958,Blue,320,1,106,01-01-2023,Week-1,Q1,2023,3128.0,749,4369,59,0.239,Swipe,Fuel,1004.87,1


In [12]:
# remove duplicate values 

len(df)
df.drop_duplicates()
len(df)

10108

In [13]:
 df.to_csv("credit_card_new")